In [170]:
import torch 
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
from torchsummary import summary
import torch.optim as optim
import matplotlib.pyplot as plt

from torchvision.models import resnet50, ResNet50_Weights

In [171]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root="dataset 2", transform=transform)
'''X_train, y_train, X_test, y_test = torch.utils.data.random_split(dataset, train=0.8, random_state=42)'''

# Define the size of the training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into training and testing datasets
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

# dataloader for batches   
train_loader = DataLoader(dataset, batch_size= 200, shuffle=True)
test_loader = DataLoader(dataset, batch_size=200, shuffle=False)


In [172]:
# move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load pre-train model: get most updated weights
model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)

# change the input layer dimension 
input_dim = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, input_dim )

# check the architecture
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [173]:
# move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# freeze layers, unfreeze some layers for fine-tuning 
for param in model.parameters():
    param.requires_grad= False

for name, param in model.named_parameters():
    if "layer4.2.conv2" in name or "layer4.2.bn2" in name:
        param.requires_grad = True

model = model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
loss_list =[]

# training loop
epoch = 10
for epoch in range(epoch):
    running_loss = 0.0
    for i, (data, label) in enumerate(train_loader): # for each batches
        data = data.to(device)
        label = label.to(device)

        # forward pass
        outputs = model(data)
        loss = criterion(outputs, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loss_list.append(loss.item())

        if i % 10 == 9:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/10], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}")
            running_loss = 0.0

plt.plot(range(epoch),loss_list)
plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

print("Training is done!")
torch.save(model.state_dict(), "model_weights.pth")

Epoch [1/5], Step [10/49], Loss: 4.8657
Epoch [1/5], Step [20/49], Loss: 4.8648


KeyboardInterrupt: 

In [ ]:
# evaluation 
model.eval()
test_loss = 0.0

with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:
        data = images.to(device)
        label = label.to(device)

        outputs = model(data)
        loss = criterion(outputs, label)
        test_loss += loss.item()

        # compute predicted class labels
        _, predicted = torch.max(outputs, dim=1)

        # update if predicted == label
        n_correct += (predicted == labels).sum().item()

test_loss = test_loss/ len(test_loader)
accuracy = n_correct/n_samples
print(f"Test loss: {test_loss:.4f}, Accruacy: {accuracy:.4f}")